# TODO:

- generate images from train dataset
- generate images from test dataset
- generate images from out of input distribution (no real airports)
- generate images for all train & test dataset (+ out of distribution ?)
- retrain yolo-nas and compare results

WITHOUT / PARTIALLY / WITH
mAP ...

# CODE

In [1]:
import json
import sys
import os
import torch
from pathlib import Path
import cv2
from torchvision import transforms
from tqdm import tqdm
import numpy as np

sys.path.append("..")

from options.train_options import TrainOptions
from models import gan_networks

/home/geoffrey.g.delhomme/miniconda3/envs/joligen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/geoffrey.g.delhomme/miniconda3/envs/joligen/lib/python3.10/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/geoffrey.g.delhomme/miniconda3/envs/joligen/lib/python3.10/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expect

In [5]:
IMAGE_DIRPATH = "../tmp/samples"
DESTINATION_DIRPATH = "../tmp/output"
# IMAGE_DIRPATH = "/mnt/data1/user_cache/geoffrey.g.delhomme/data/2023_12/coco-1024x750-augmented/train/images_origin"
# DESTINATION_DIRPATH = "/mnt/data1/user_cache/geoffrey.g.delhomme/data/2023_12/coco-1024x750-augmented/train/images"
if IMAGE_DIRPATH is not None:
    IMAGE_FILEPATHS = [f.as_posix() for f in Path(IMAGE_DIRPATH).iterdir() if f.suffix in ['.png', '.jpg', '.jpeg']]
MODEL_FILEPATH = "/home/geoffrey.g.delhomme/projects/joliGEN/checkpoints/synthetic2real_mask_online/20_net_G_A.pth"
# MODEL_FILEPATH = "/home/geoffrey.g.delhomme/projects/joliGEN/checkpoints/synthetic2real_mask_online/39_net_G_A.pth"
CONFIG_FILEPATH = "/home/geoffrey.g.delhomme/projects/joliGEN/examples/example_gan_synthetic2real_mask_online.json"
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 752
GPUID = 0 # None: cpu else gpu id
CONCAT = True

In [6]:
cpu = GPUID is None
gpuid = GPUID

def get_z_random(batch_size=1, nz=8, random_type="gauss"):
    if random_type == "uni":
        z = torch.rand(batch_size, nz) * 2.0 - 1.0
    elif random_type == "gauss":
        z = torch.randn(batch_size, nz)
    return z.detach()

# load model
with open(CONFIG_FILEPATH, 'r') as f:
    config = json.load(f)
opt = TrainOptions().parse_json(config, set_device=False)
if opt.model_multimodal:
    opt.model_input_nc += opt.train_mm_nz
opt.jg_dir = Path('..').resolve().as_posix()
if not cpu:
    device = torch.device("cuda:" + str(gpuid))
else:
    device = torch.device("cpu")
print(f"Device: {device}")
model = gan_networks.define_G(**vars(opt))
model.eval()
model.load_state_dict(
    torch.load(MODEL_FILEPATH, map_location=device)
)
model = model.to(device)

# preprocess
tranlist = [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]
tran = transforms.Compose(tranlist)
for image_filepath in tqdm(IMAGE_FILEPATHS):
    image_filepath = Path(image_filepath)
    # load image
    img_orig = cv2.imread(image_filepath.as_posix())
    img = cv2.cvtColor(img_orig.copy(), cv2.COLOR_BGR2RGB)
    orig_height, orig_width, _ = img.shape
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    # preprocess
    img_tensor = tran(img).to(device)

    if opt.model_multimodal:
        z_random = get_z_random(batch_size=1, nz=opt.train_mm_nz)
        z_random = z_random.to(device)
        # print('z_random shape=', self.z_random.shape)
        z_real = z_random.view(z_random.size(0), z_random.size(1), 1, 1).expand(
            z_random.size(0),
            z_random.size(1),
            img_tensor.size(1),
            img_tensor.size(2),
        )
        img_tensor = torch.cat([img_tensor.unsqueeze(0), z_real], 1)
    else:
        img_tensor = img_tensor.unsqueeze(0)

    # run through model
    out_tensor = model(img_tensor)[0].detach()

    # post-processing
    out_img = out_tensor.data.cpu().float().numpy()
    out_img = (np.transpose(out_img, (1, 2, 0)) + 1) / 2.0 * 255.0
    # print(out_img)
    out_img = cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR)
    out_img = cv2.resize(out_img, (orig_width, orig_height), interpolation=cv2.INTER_CUBIC)
    if CONCAT:
        out_img = np.concatenate((img_orig, out_img), axis=1)
    out_image_filepath = Path(DESTINATION_DIRPATH) / image_filepath.name
    out_image_filepath.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(out_image_filepath.as_posix(), out_img)

Device: cuda:0
initialize network with normal


100%|██████████| 8/8 [00:02<00:00,  2.98it/s]


: 